In [1]:
import praw
from textblob import TextBlob
import pandas as pd

In [2]:
client_id = ''
client_secret = ''
user_agent = ''

In [3]:
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)


In [4]:
def clean_text(text):
    return text.replace('\n', ' ').strip()

In [5]:
def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [14]:
import pandas as pd
from datetime import datetime

def crawl_and_get_posts(subreddit_name, max_posts, target_flair=None):
    subreddit = reddit.subreddit(subreddit_name)

    data = []
    for post in subreddit.hot(limit=max_posts):
        if post.stickied:
            continue

        if target_flair is None or (post.link_flair_text and target_flair.lower() == post.link_flair_text.lower()):
                cleaned_title = clean_text(post.title)
                cleaned_content = clean_text(post.selftext)
                
                for comment in post.comments:
                    if isinstance(comment, praw.models.Comment):
                        cleaned_comment = clean_text(comment.body)
                        
                        sentiment = get_sentiment(cleaned_title + ' ' + cleaned_content + ' ' + cleaned_comment)
                        post_date = datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')
                        comment_date = datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
                        
                        data.append({'post_title': cleaned_title,
                                    'post_content': cleaned_content,
                                    'post_comment': cleaned_comment,
                                    'subreddit': subreddit_name,
                                    'sentiment': sentiment,
                                    'post_date': post_date,
                                    'post_link': post.url,
                                    'comment_date': comment_date,
                                    'comment_link': comment.permalink})
    return pd.DataFrame(data)


In [15]:
#Crawl subreddit of mental health, flair: Needs Support
subreddit_name1 = 'mentalhealth'
max_post1 = 20000
tf1 = 'Need Support'
df1 = crawl_and_get_posts(subreddit_name1, max_post1, tf1)
df1

,post_title,post_content,post_comment,subreddit,sentiment,post_date,post_link,comment_date,comment_link
0,How do I get over sexual jealousy,I know how pathetic it sounds but I really don...,1. Don't normalize what other people do or wha...,mentalhealth,negative,2024-03-04 23:10:43,https://www.reddit.com/r/mentalhealth/comments...,2024-03-04 23:44:26,/r/mentalhealth/comments/1b6pc2l/how_do_i_get_...
1,How do I get over sexual jealousy,I know how pathetic it sounds but I really don...,Never compare yourself to anybody else. Your l...,mentalhealth,negative,2024-03-04 23:10:43,https://www.reddit.com/r/mentalhealth/comments...,2024-03-05 00:28:51,/r/mentalhealth/comments/1b6pc2l/how_do_i_get_...
2,How do I get over sexual jealousy,I know how pathetic it sounds but I really don...,I can relate but I do like to remind myself th...,mentalhealth,negative,2024-03-04 23:10:43,https://www.reddit.com/r/mentalhealth/comments...,2024-03-04 23:46:40,/r/mentalhealth/comments/1b6pc2l/how_do_i_get_...
3,How do I get over sexual jealousy,I know how pathetic it sounds but I really don...,I'm almost certain most people lie about the a...,mentalhealth,positive,2024-03-04 23:10:43,https://www.reddit.com/r/mentalhealth/comments...,2024-03-05 00:02:33,/r/mentalhealth/comments/1b6pc2l/how_do_i_get_...
4,How do I get over sexual jealousy,I know how pathetic it sounds but I really don...,Find yourself a nice woman and try to have a r...,mentalhealth,negative,2024-03-04 23:10:43,https://www.reddit.com/r/mentalhealth/comments...,2024-03-05 00:05:27,/r/mentalhealth/comments/1b6pc2l/how_do_i_get_...
...,...,...,...,...,...,...,...,...,...
910,I need to change myself ASAP. How do I do this...,I need to improve a Good 95% of myself ASAP. I...,Cognitive behavioral therapy might be a good o...,mentalhealth,positive,2024-03-01 04:36:26,https://www.reddit.com/r/mentalhealth/comments...,2024-03-01 05:32:15,/r/mentalhealth/comments/1b3ljwa/i_need_to_cha...
911,I need to change myself ASAP. How do I do this...,I need to improve a Good 95% of myself ASAP. I...,Here are some of the non-negotiables that you ...,mentalhealth,positive,2024-03-01 04:36:26,https://www.reddit.com/r/mentalhealth/comments...,2024-03-01 06:19:18,/r/mentalhealth/comments/1b3ljwa/i_need_to_cha...
912,Burn out anecdote,"Hi, so I (25F) am feeling very dissociated and...",You can do some meditation to calm your mind. ...,mentalhealth,positive,2024-03-01 09:20:51,https://www.reddit.com/r/mentalhealth/comments...,2024-03-01 09:45:08,/r/mentalhealth/comments/1b3q4j3/burn_out_anec...
913,Burn out anecdote,"Hi, so I (25F) am feeling very dissociated and...",Take out a few minutes from your busy schedule...,mentalhealth,positive,2024-03-01 09:20:51,https://www.reddit.com/r/mentalhealth/comments...,2024-03-02 21:29:30,/r/mentalhealth/comments/1b3q4j3/burn_out_anec...


In [16]:
# Crawl subreddit of social anxiety, 
subreddit_name2 = 'socialanxiety'
max_post2 = 30000
df2 = crawl_and_get_posts(subreddit_name2, max_post2)
df2

,post_title,post_content,post_comment,subreddit,sentiment,post_date,post_link,comment_date,comment_link
0,"i'll probably never have a girlfriend, and i d...","i know some people might pop in here like ""the...",i think i'm only getting a gf if one girl ever...,socialanxiety,positive,2024-03-05 04:04:07,https://www.reddit.com/r/socialanxiety/comment...,2024-03-05 04:44:40,/r/socialanxiety/comments/1b6vx8o/ill_probably...
1,"i'll probably never have a girlfriend, and i d...","i know some people might pop in here like ""the...","I feel you man, I think I’m on the spectrum sl...",socialanxiety,positive,2024-03-05 04:04:07,https://www.reddit.com/r/socialanxiety/comment...,2024-03-05 05:20:55,/r/socialanxiety/comments/1b6vx8o/ill_probably...
2,"i'll probably never have a girlfriend, and i d...","i know some people might pop in here like ""the...",You need to replace your desire for women/ a r...,socialanxiety,positive,2024-03-05 04:04:07,https://www.reddit.com/r/socialanxiety/comment...,2024-03-05 05:46:22,/r/socialanxiety/comments/1b6vx8o/ill_probably...
3,"i'll probably never have a girlfriend, and i d...","i know some people might pop in here like ""the...",I feel exactly the same as you and I am 40... ...,socialanxiety,positive,2024-03-05 04:04:07,https://www.reddit.com/r/socialanxiety/comment...,2024-03-05 05:45:04,/r/socialanxiety/comments/1b6vx8o/ill_probably...
4,"i'll probably never have a girlfriend, and i d...","i know some people might pop in here like ""the...","Not sure if it’s okay to share resources here,...",socialanxiety,positive,2024-03-05 04:04:07,https://www.reddit.com/r/socialanxiety/comment...,2024-03-05 06:21:40,/r/socialanxiety/comments/1b6vx8o/ill_probably...
...,...,...,...,...,...,...,...,...,...
4183,Why do I overthink?,Why can’t I just talk to someone bedside me an...,yes i used to feel so bad for myself because o...,socialanxiety,positive,2024-02-17 22:44:50,https://www.reddit.com/r/socialanxiety/comment...,2024-02-18 08:17:30,/r/socialanxiety/comments/1atelyl/why_do_i_ove...
4184,Anxious about my presentations,I'm a French exchange student in Mexico. I'm r...,good luck with your presentations!! try to gri...,socialanxiety,positive,2024-02-17 22:40:57,https://www.reddit.com/r/socialanxiety/comment...,2024-02-18 08:24:09,/r/socialanxiety/comments/1ateir7/anxious_abou...
4185,How to socialize when people have always laugh...,When I was roughly 15-16 years old I’d occasio...,"people r harsh it’s just the reality of it, I’...",socialanxiety,positive,2024-02-17 15:08:49,https://www.reddit.com/r/socialanxiety/comment...,2024-02-17 15:33:30,/r/socialanxiety/comments/1at427k/how_to_socia...
4186,When to taper off,When its the time we should start tapering off...,[deleted],socialanxiety,neutral,2024-02-17 12:06:23,https://www.reddit.com/r/socialanxiety/comment...,2024-02-17 12:28:31,/r/socialanxiety/comments/1at0h5y/when_to_tape...


In [17]:
# Crawl subreddit of mental health support 
subreddit_name3 = 'MentalHealthSupport'
max_post3 = 40000
df3 = crawl_and_get_posts(subreddit_name3, max_post3)
df3

,post_title,post_content,post_comment,subreddit,sentiment,post_date,post_link,comment_date,comment_link
0,I feel lost,"I feel lost, I guess. I grew up very introvert...",I definitely get where you’re coming from and ...,MentalHealthSupport,negative,2024-03-05 04:31:29,https://www.reddit.com/r/MentalHealthSupport/c...,2024-03-05 05:16:42,/r/MentalHealthSupport/comments/1b6wgdq/i_feel...
1,A guide to better mental health. Everything I ...,"Therapy is excellent, no discrediting it, but ...",Feel free to ask anything too have alot more t...,MentalHealthSupport,positive,2024-03-05 02:24:36,https://www.reddit.com/r/MentalHealthSupport/c...,2024-03-05 05:13:39,/r/MentalHealthSupport/comments/1b6tspm/a_guid...
2,how do i know that i need a therapist?,i dont know what it is? i don’t know if it’s a...,Try not to think of a therapist as something y...,MentalHealthSupport,positive,2024-03-05 05:16:20,https://www.reddit.com/r/MentalHealthSupport/c...,2024-03-05 05:39:06,/r/MentalHealthSupport/comments/1b6xar7/how_do...
3,how do i know that i need a therapist?,i dont know what it is? i don’t know if it’s a...,Don't really remember where I heard this (Firs...,MentalHealthSupport,positive,2024-03-05 05:16:20,https://www.reddit.com/r/MentalHealthSupport/c...,2024-03-05 06:03:31,/r/MentalHealthSupport/comments/1b6xar7/how_do...
4,I need help.,I don‘t know what to do. I have people around ...,"Hey! I'm sorry you're going through that, I do...",MentalHealthSupport,positive,2024-03-05 04:39:44,https://www.reddit.com/r/MentalHealthSupport/c...,2024-03-05 06:42:56,/r/MentalHealthSupport/comments/1b6wltg/i_need...
...,...,...,...,...,...,...,...,...,...
2007,I don't want to live anymore because of my pan...,In 2020 on last school day I experienced my fi...,I'm sorry to hear this. It honestly feels real...,MentalHealthSupport,positive,2022-12-03 12:10:49,https://www.reddit.com/r/MentalHealthSupport/c...,2022-12-03 21:39:35,/r/MentalHealthSupport/comments/zbevjt/i_dont_...
2008,I don't want to live anymore because of my pan...,In 2020 on last school day I experienced my fi...,Although I'm not entirely certain if you're st...,MentalHealthSupport,positive,2022-12-03 12:10:49,https://www.reddit.com/r/MentalHealthSupport/c...,2023-10-03 21:34:07,/r/MentalHealthSupport/comments/zbevjt/i_dont_...
2009,How do I control my PTSD nightmares and flashb...,so for some context. I was at the Manchester A...,"I don’t have advice, but I do want to send my ...",MentalHealthSupport,positive,2022-12-01 04:47:05,https://www.reddit.com/r/MentalHealthSupport/c...,2022-12-01 05:39:40,/r/MentalHealthSupport/comments/z9enoi/how_do_...
2010,I don’t even have anything to live for anymore,Nobody likes me and I lost the one person who ...,Are you still there? Are you ok?,MentalHealthSupport,positive,2022-11-30 02:31:12,https://www.reddit.com/r/MentalHealthSupport/c...,2022-11-30 09:05:56,/r/MentalHealthSupport/comments/z8ehb4/i_dont_...


In [18]:
# Crawl subreddit of depression 
subreddit_name4 = 'depression'
max_post4 = 20000
df4 = crawl_and_get_posts(subreddit_name4, max_post4)
df4

,post_title,post_content,post_comment,subreddit,sentiment,post_date,post_link,comment_date,comment_link
0,Depression makes me gross,Sometimes it’s too hard to brush my teeth or m...,God I feel you so hard on this. I spent the ot...,depression,negative,2024-03-04 18:11:34,https://www.reddit.com/r/depression/comments/1...,2024-03-04 18:31:30,/r/depression/comments/1b6hto5/depression_make...
1,Depression makes me gross,Sometimes it’s too hard to brush my teeth or m...,You don't need to be ashamed. This is a real s...,depression,positive,2024-03-04 18:11:34,https://www.reddit.com/r/depression/comments/1...,2024-03-04 19:31:06,/r/depression/comments/1b6hto5/depression_make...
2,Depression makes me gross,Sometimes it’s too hard to brush my teeth or m...,You should give yourself more credit for swimm...,depression,positive,2024-03-04 18:11:34,https://www.reddit.com/r/depression/comments/1...,2024-03-04 22:42:17,/r/depression/comments/1b6hto5/depression_make...
3,Depression makes me gross,Sometimes it’s too hard to brush my teeth or m...,No one understands this!!!!! I’ve been serious...,depression,positive,2024-03-04 18:11:34,https://www.reddit.com/r/depression/comments/1...,2024-03-04 22:03:44,/r/depression/comments/1b6hto5/depression_make...
4,Depression makes me gross,Sometimes it’s too hard to brush my teeth or m...,The last sentence though - well done OP! Litt...,depression,negative,2024-03-04 18:11:34,https://www.reddit.com/r/depression/comments/1...,2024-03-04 20:09:31,/r/depression/comments/1b6hto5/depression_make...
...,...,...,...,...,...,...,...,...,...
1551,"Do things like exercising regularly, eating he...",Everyone always tells me to do these three thi...,"It helps me feel better physically, which help...",depression,positive,2024-02-29 22:37:33,https://www.reddit.com/r/depression/comments/1...,2024-03-01 09:34:12,/r/depression/comments/1b3doyx/do_things_like_...
1552,"Do things like exercising regularly, eating he...",Everyone always tells me to do these three thi...,"I don't think so, it sounds like the ol' prote...",depression,negative,2024-02-29 22:37:33,https://www.reddit.com/r/depression/comments/1...,2024-03-01 14:22:18,/r/depression/comments/1b3doyx/do_things_like_...
1553,Vicious Cycle,"Hi all, I (24m) have been dealing with depre...",I don’t have any advice but I’m wishing you th...,depression,positive,2024-03-01 05:13:45,https://www.reddit.com/r/depression/comments/1...,2024-03-01 06:10:42,/r/depression/comments/1b3m8xs/vicious_cycle/k...
1554,You can’t win anymore,The prevailing culture is one that tears peopl...,I agree. I cannot even explore any sort of ide...,depression,positive,2024-02-29 17:36:11,https://www.reddit.com/r/depression/comments/1...,2024-02-29 19:16:09,/r/depression/comments/1b364vp/you_cant_win_an...


In [19]:
# Crawl subreddit of mentalillness/Action needed 
subreddit_name5 = 'mentalillness'
max_post5 = 20000
tf5 = 'Advice Needed' 
df5 = crawl_and_get_posts(subreddit_name5, max_post5, tf5)
df5

,post_title,post_content,post_comment,subreddit,sentiment,post_date,post_link,comment_date,comment_link
0,Need help for my grandmother suffering from va...,"Hello, my grandmother is suffering from vascul...",I’m so sorry you’re going through this. Vascul...,mentalillness,positive,2024-03-04 22:27:47,https://www.reddit.com/r/mentalillness/comment...,2024-03-05 01:28:34,/r/mentalillness/comments/1b6o86w/need_help_fo...
1,Someone please help me out for my current situ...,My brother who is in his early 20s recently ha...,"That sounds *really* scary, painful, and frust...",mentalillness,positive,2024-03-04 21:26:35,https://www.reddit.com/r/mentalillness/comment...,2024-03-05 01:42:19,/r/mentalillness/comments/1b6mp4q/someone_plea...
2,i think theyre after me,so this predicament started about november of ...,"hey, I'm so sorry you are going through this. ...",mentalillness,negative,2024-03-04 21:09:48,https://www.reddit.com/r/mentalillness/comment...,2024-03-04 22:43:33,/r/mentalillness/comments/1b6m9t5/i_think_they...
3,It was fake at first.,"Help me please, I don't know what it is Hey,...",I seriously doubt you gave yourself a mental i...,mentalillness,negative,2024-03-04 13:19:20,https://www.reddit.com/r/mentalillness/comment...,2024-03-04 14:05:24,/r/mentalillness/comments/1b6apcc/it_was_fake_...
4,It was fake at first.,"Help me please, I don't know what it is Hey,...","No, you can't make yourself mentally ill in th...",mentalillness,negative,2024-03-04 13:19:20,https://www.reddit.com/r/mentalillness/comment...,2024-03-04 14:05:59,/r/mentalillness/comments/1b6apcc/it_was_fake_...
...,...,...,...,...,...,...,...,...,...
534,hello,has anyone here overcome depression without th...,A good therapist should equip you with the men...,mentalillness,positive,2024-01-29 18:19:18,https://www.reddit.com/r/mentalillness/comment...,2024-01-30 08:10:30,/r/mentalillness/comments/1ae241e/hello/kk8ddig/
535,I (19F) am so lost. I cant bring myself to so ...,I really dont know what to do right now. I am ...,Please don't hide it from your parents any mor...,mentalillness,negative,2024-01-29 08:20:48,https://www.reddit.com/r/mentalillness/comment...,2024-01-29 09:52:04,/r/mentalillness/comments/1adqegs/i_19f_am_so_...
536,I (19F) am so lost. I cant bring myself to so ...,I really dont know what to do right now. I am ...,Have you tried going out for a walk without yo...,mentalillness,negative,2024-01-29 08:20:48,https://www.reddit.com/r/mentalillness/comment...,2024-01-29 10:11:07,/r/mentalillness/comments/1adqegs/i_19f_am_so_...
537,I (19F) am so lost. I cant bring myself to so ...,I really dont know what to do right now. I am ...,This really sounds like depression but I am no...,mentalillness,negative,2024-01-29 08:20:48,https://www.reddit.com/r/mentalillness/comment...,2024-01-29 11:11:29,/r/mentalillness/comments/1adqegs/i_19f_am_so_...


In [20]:
# Crawl subreddit of Anxiety/Advice needed 
subreddit_name6 = 'Anxiety'
max_post6 = 20000
tf6 = 'Advice Needed' 
df6 = crawl_and_get_posts(subreddit_name6, max_post6, tf6)
df6

,post_title,post_content,post_comment,subreddit,sentiment,post_date,post_link,comment_date,comment_link
0,Am I doing meditation/mindfulness wrong?,I have GAD and panic disorder. I have been try...,"It’s definitely normal, especially when starti...",Anxiety,negative,2024-03-05 04:35:58,https://www.reddit.com/r/Anxiety/comments/1b6w...,2024-03-05 05:06:27,/r/Anxiety/comments/1b6wjed/am_i_doing_meditat...
1,Am I doing meditation/mindfulness wrong?,I have GAD and panic disorder. I have been try...,Definitely normal but don’t get discouraged! M...,Anxiety,positive,2024-03-05 04:35:58,https://www.reddit.com/r/Anxiety/comments/1b6w...,2024-03-05 05:28:42,/r/Anxiety/comments/1b6wjed/am_i_doing_meditat...
2,Am I doing meditation/mindfulness wrong?,I have GAD and panic disorder. I have been try...,I'm far from an expert on it but this is what ...,Anxiety,positive,2024-03-05 04:35:58,https://www.reddit.com/r/Anxiety/comments/1b6w...,2024-03-05 06:29:08,/r/Anxiety/comments/1b6wjed/am_i_doing_meditat...
3,Severe anxiety for first time in life at 29? (...,Update: Just want to say thank you so much. Ea...,This happened to me last year in January. I ha...,Anxiety,positive,2024-03-04 22:21:39,https://www.reddit.com/r/Anxiety/comments/1b6o...,2024-03-04 22:38:07,/r/Anxiety/comments/1b6o2ou/severe_anxiety_for...
4,Severe anxiety for first time in life at 29? (...,Update: Just want to say thank you so much. Ea...,It tends to develope incredibly slowly from ce...,Anxiety,positive,2024-03-04 22:21:39,https://www.reddit.com/r/Anxiety/comments/1b6o...,2024-03-04 22:30:00,/r/Anxiety/comments/1b6o2ou/severe_anxiety_for...
...,...,...,...,...,...,...,...,...,...
451,Does anyone else worry about not being worried?,I don't really know how to explain this. Esse...,"100 %, I'll be in a good place feeling really ...",Anxiety,positive,2024-02-28 22:26:30,https://www.reddit.com/r/Anxiety/comments/1b2j...,2024-02-29 00:16:30,/r/Anxiety/comments/1b2ji5f/does_anyone_else_w...
452,Help with alcohol,I am going through a rough divorce and some ot...,Exercise really helped me. I just go to the gy...,Anxiety,negative,2024-02-29 01:46:43,https://www.reddit.com/r/Anxiety/comments/1b2o...,2024-02-29 02:25:43,/r/Anxiety/comments/1b2o6i0/help_with_alcohol/...
453,Help with alcohol,I am going through a rough divorce and some ot...,Amitriptyline makes you fall asleep extremely ...,Anxiety,negative,2024-02-29 01:46:43,https://www.reddit.com/r/Anxiety/comments/1b2o...,2024-02-29 02:33:42,/r/Anxiety/comments/1b2o6i0/help_with_alcohol/...
454,Prolonged fight or flight?,"So I've struggled with anxiety for years, but ...",I’m feeling like this at the moment I’m chilli...,Anxiety,negative,2024-02-28 22:00:09,https://www.reddit.com/r/Anxiety/comments/1b2i...,2024-02-28 22:19:30,/r/Anxiety/comments/1b2itnn/prolonged_fight_or...


In [21]:
# Crawl subreddit of bipolar 
subreddit_name7 = 'bipolar'
max_post7 = 20000
tf7 = 'Just Sharing' 
df7 = crawl_and_get_posts(subreddit_name7, max_post7, tf7)
df7

,post_title,post_content,post_comment,subreddit,sentiment,post_date,post_link,comment_date,comment_link
0,I took a shower,You know I am trying to figure out why a 65 ye...,Thanks for posting on /r/bipolar! Please ...,bipolar,positive,2024-03-04 21:51:50,https://www.reddit.com/r/bipolar/comments/1b6n...,2024-03-04 21:51:51,/r/bipolar/comments/1b6nbk0/i_took_a_shower/kt...
1,I took a shower,You know I am trying to figure out why a 65 ye...,yeah i hate showering and have no reason to ha...,bipolar,positive,2024-03-04 21:51:50,https://www.reddit.com/r/bipolar/comments/1b6n...,2024-03-05 00:25:58,/r/bipolar/comments/1b6nbk0/i_took_a_shower/kt...
2,I took a shower,You know I am trying to figure out why a 65 ye...,Reminded me of a [post](https://www.reddit.com...,bipolar,positive,2024-03-04 21:51:50,https://www.reddit.com/r/bipolar/comments/1b6n...,2024-03-05 02:54:03,/r/bipolar/comments/1b6nbk0/i_took_a_shower/kt...
3,I took a shower,You know I am trying to figure out why a 65 ye...,I hear you! As soon as I saw the title I wante...,bipolar,positive,2024-03-04 21:51:50,https://www.reddit.com/r/bipolar/comments/1b6n...,2024-03-05 03:26:54,/r/bipolar/comments/1b6nbk0/i_took_a_shower/kt...
4,I took a shower,You know I am trying to figure out why a 65 ye...,Maybe try to make it less of a chore and more ...,bipolar,positive,2024-03-04 21:51:50,https://www.reddit.com/r/bipolar/comments/1b6n...,2024-03-05 03:18:37,/r/bipolar/comments/1b6nbk0/i_took_a_shower/kt...
...,...,...,...,...,...,...,...,...,...
882,Dead eyes,Looking at old photos I took in an attempt to ...,Dead eyes is how I know I’m depressed it’s kin...,bipolar,negative,2024-02-21 23:43:17,https://www.reddit.com/r/bipolar/comments/1awr...,2024-02-22 06:44:53,/r/bipolar/comments/1awrbp1/dead_eyes/krkp9zc/
883,Dead eyes,Looking at old photos I took in an attempt to ...,"Funny you say this, my husband makes comments ...",bipolar,positive,2024-02-21 23:43:17,https://www.reddit.com/r/bipolar/comments/1awr...,2024-02-22 02:00:21,/r/bipolar/comments/1awrbp1/dead_eyes/krjon1f/
884,Two years since hit and run,So back in 2022 I crawled under a red crv caus...,Thanks for posting on /r/bipolar! Please ...,bipolar,positive,2024-02-21 21:46:04,https://www.reddit.com/r/bipolar/comments/1awo...,2024-02-21 21:46:05,/r/bipolar/comments/1awodf2/two_years_since_hi...
885,Two years since hit and run,So back in 2022 I crawled under a red crv caus...,"Dawg, that is insane levels of mania. Kudos to...",bipolar,positive,2024-02-21 21:46:04,https://www.reddit.com/r/bipolar/comments/1awo...,2024-02-21 22:52:58,/r/bipolar/comments/1awodf2/two_years_since_hi...


In [22]:
# Crawl subreddit of ptsd 
subreddit_name8 = 'ptsd'
max_post8 = 15000
tf8 = 'Support' 
df8 = crawl_and_get_posts(subreddit_name8, max_post8, tf8)
df8

,post_title,post_content,post_comment,subreddit,sentiment,post_date,post_link,comment_date,comment_link
0,DAE have somatic flashbacks?,Maybe an odd post but I've noticed a major sym...,*r/ptsd has generated this automated response ...,ptsd,positive,2024-03-04 19:42:25,https://www.reddit.com/r/ptsd/comments/1b6k3jx...,2024-03-04 19:42:26,/r/ptsd/comments/1b6k3jx/dae_have_somatic_flas...
1,DAE have somatic flashbacks?,Maybe an odd post but I've noticed a major sym...,Jesus somatic flashbacks are my worst symptom ...,ptsd,positive,2024-03-04 19:42:25,https://www.reddit.com/r/ptsd/comments/1b6k3jx...,2024-03-04 21:54:08,/r/ptsd/comments/1b6k3jx/dae_have_somatic_flas...
2,DAE have somatic flashbacks?,Maybe an odd post but I've noticed a major sym...,Surprisingly physical flashbacks were my first...,ptsd,positive,2024-03-04 19:42:25,https://www.reddit.com/r/ptsd/comments/1b6k3jx...,2024-03-05 01:34:41,/r/ptsd/comments/1b6k3jx/dae_have_somatic_flas...
3,i’m scared i’ll never going to be happy,i’m scared i’ll never be happy. i have no inte...,*r/ptsd has generated this automated response ...,ptsd,positive,2024-03-05 02:09:26,https://www.reddit.com/r/ptsd/comments/1b6tgqk...,2024-03-05 02:09:27,/r/ptsd/comments/1b6tgqk/im_scared_ill_never_g...
4,"it's never going to get better, is it ?",tw sexual assault The traumatic events that ga...,*r/ptsd has generated this automated response ...,ptsd,positive,2024-03-04 15:45:55,https://www.reddit.com/r/ptsd/comments/1b6e3tg...,2024-03-04 15:45:55,/r/ptsd/comments/1b6e3tg/its_never_going_to_ge...
...,...,...,...,...,...,...,...,...,...
1064,I still have PTSD from high school despite gra...,I was bullied to hell and back. School did no...,I can relate. A large chunk of my trauma is fr...,ptsd,negative,2024-01-27 02:06:30,https://www.reddit.com/r/ptsd/comments/1abzndy...,2024-01-27 02:34:56,/r/ptsd/comments/1abzndy/i_still_have_ptsd_fro...
1065,I still have PTSD from high school despite gra...,I was bullied to hell and back. School did no...,"that sounds terrible, it’s like psychological ...",ptsd,negative,2024-01-27 02:06:30,https://www.reddit.com/r/ptsd/comments/1abzndy...,2024-01-27 02:22:56,/r/ptsd/comments/1abzndy/i_still_have_ptsd_fro...
1066,I still have PTSD from high school despite gra...,I was bullied to hell and back. School did no...,I could’ve written the first paragraph - scary...,ptsd,positive,2024-01-27 02:06:30,https://www.reddit.com/r/ptsd/comments/1abzndy...,2024-01-27 17:48:57,/r/ptsd/comments/1abzndy/i_still_have_ptsd_fro...
1067,I still have PTSD from high school despite gra...,I was bullied to hell and back. School did no...,Have you considered working at clearing the pt...,ptsd,positive,2024-01-27 02:06:30,https://www.reddit.com/r/ptsd/comments/1abzndy...,2024-01-27 16:04:01,/r/ptsd/comments/1abzndy/i_still_have_ptsd_fro...


In [24]:
# Crawl subreddit of EatingDisorders 
subreddit_name9 = 'EatingDisorders'
max_post9 = 18000
df9 = crawl_and_get_posts(subreddit_name9, max_post9)
df9

,post_title,post_content,post_comment,subreddit,sentiment,post_date,post_link,comment_date,comment_link
0,I want to get worse,Does anyone else feel this way??? I've been e...,I totally get this. I miss the days when it ma...,EatingDisorders,negative,2024-03-05 01:31:25,https://www.reddit.com/r/EatingDisorders/comme...,2024-03-05 07:19:59,/r/EatingDisorders/comments/1b6smh6/i_want_to_...
1,I want to get worse,Does anyone else feel this way??? I've been e...,I totally understand.,EatingDisorders,negative,2024-03-05 01:31:25,https://www.reddit.com/r/EatingDisorders/comme...,2024-03-05 07:00:01,/r/EatingDisorders/comments/1b6smh6/i_want_to_...
2,I want to get worse,Does anyone else feel this way??? I've been e...,I go through periods of this. Especially when ...,EatingDisorders,negative,2024-03-05 01:31:25,https://www.reddit.com/r/EatingDisorders/comme...,2024-03-05 07:30:09,/r/EatingDisorders/comments/1b6smh6/i_want_to_...
3,I want to get worse,Does anyone else feel this way??? I've been e...,I am feeling just like this atm. My main probl...,EatingDisorders,negative,2024-03-05 01:31:25,https://www.reddit.com/r/EatingDisorders/comme...,2024-03-05 05:25:01,/r/EatingDisorders/comments/1b6smh6/i_want_to_...
4,Have you guys done tube feeding?,I think I might have to do it and I want to kn...,"Hi, this is my experience so i hope it will he...",EatingDisorders,positive,2024-03-05 04:04:25,https://www.reddit.com/r/EatingDisorders/comme...,2024-03-05 04:22:30,/r/EatingDisorders/comments/1b6vxgf/have_you_g...
...,...,...,...,...,...,...,...,...,...
1792,I think my ED is fooling with me,As if it wouldn’t be hard enough to continue r...,I think it’s pretty common for hair to thin du...,EatingDisorders,negative,2023-12-18 15:28:22,https://www.reddit.com/r/EatingDisorders/comme...,2023-12-21 22:42:12,/r/EatingDisorders/comments/18lbdl5/i_think_my...
1793,"Binge eating, Weight, help!",I am a 29 year old female. I have struggled wi...,I guess I’d need more info about your daily fo...,EatingDisorders,positive,2023-12-18 19:39:17,https://www.reddit.com/r/EatingDisorders/comme...,2023-12-19 04:42:01,/r/EatingDisorders/comments/18lhgj0/binge_eati...
1794,"Binge eating, Weight, help!",I am a 29 year old female. I have struggled wi...,You have for sure messed up your hunger signal...,EatingDisorders,positive,2023-12-18 19:39:17,https://www.reddit.com/r/EatingDisorders/comme...,2023-12-19 12:39:13,/r/EatingDisorders/comments/18lhgj0/binge_eati...
1795,"Binge eating, Weight, help!",I am a 29 year old female. I have struggled wi...,[removed],EatingDisorders,positive,2023-12-18 19:39:17,https://www.reddit.com/r/EatingDisorders/comme...,2023-12-19 15:09:47,/r/EatingDisorders/comments/18lhgj0/binge_eati...


In [26]:
# Crawl subreddit of EatingDisorders 
subreddit_name10 = 'schizophrenia'
max_post10 =18000
tf10 = 'Seeking Support'
df10 = crawl_and_get_posts(subreddit_name10, max_post10)
df10

,post_title,post_content,post_comment,subreddit,sentiment,post_date,post_link,comment_date,comment_link
0,I feel absolutely depressed,I don’t feel successful. I’m just someone who’...,I hope you start feeling better soon. Just kno...,schizophrenia,positive,2024-03-05 02:16:05,https://www.reddit.com/r/schizophrenia/comment...,2024-03-05 02:41:48,/r/schizophrenia/comments/1b6tlzc/i_feel_absol...
1,I feel absolutely depressed,I don’t feel successful. I’m just someone who’...,"As long as you're trying, there will be better...",schizophrenia,positive,2024-03-05 02:16:05,https://www.reddit.com/r/schizophrenia/comment...,2024-03-05 02:54:56,/r/schizophrenia/comments/1b6tlzc/i_feel_absol...
2,I feel absolutely depressed,I don’t feel successful. I’m just someone who’...,"You do deserve happiness, it isn't your fault ...",schizophrenia,positive,2024-03-05 02:16:05,https://www.reddit.com/r/schizophrenia/comment...,2024-03-05 02:44:35,/r/schizophrenia/comments/1b6tlzc/i_feel_absol...
3,I feel absolutely depressed,I don’t feel successful. I’m just someone who’...,Everyone has purpose in life. Society puts pre...,schizophrenia,positive,2024-03-05 02:16:05,https://www.reddit.com/r/schizophrenia/comment...,2024-03-05 04:08:39,/r/schizophrenia/comments/1b6tlzc/i_feel_absol...
4,I feel absolutely depressed,I don’t feel successful. I’m just someone who’...,I have a similar experience. Having become dis...,schizophrenia,positive,2024-03-05 02:16:05,https://www.reddit.com/r/schizophrenia/comment...,2024-03-05 03:57:49,/r/schizophrenia/comments/1b6tlzc/i_feel_absol...
...,...,...,...,...,...,...,...,...,...
4309,Thought Projection,"Hello, as a Schizophrenic one of my only illus...",For almost a year I thought my partner was in ...,schizophrenia,positive,2024-02-15 00:23:45,https://www.reddit.com/r/schizophrenia/comment...,2024-02-15 02:51:35,/r/schizophrenia/comments/1ar2e2a/thought_proj...
4310,Thought Projection,"Hello, as a Schizophrenic one of my only illus...","One thing that could be helpful, is if people ...",schizophrenia,positive,2024-02-15 00:23:45,https://www.reddit.com/r/schizophrenia/comment...,2024-02-17 02:47:02,/r/schizophrenia/comments/1ar2e2a/thought_proj...
4311,Thought Projection,"Hello, as a Schizophrenic one of my only illus...",[removed],schizophrenia,positive,2024-02-15 00:23:45,https://www.reddit.com/r/schizophrenia/comment...,2024-02-15 00:34:53,/r/schizophrenia/comments/1ar2e2a/thought_proj...
4312,Thought Projection,"Hello, as a Schizophrenic one of my only illus...",I have that problem too- it’s just what it is ...,schizophrenia,positive,2024-02-15 00:23:45,https://www.reddit.com/r/schizophrenia/comment...,2024-02-15 07:59:11,/r/schizophrenia/comments/1ar2e2a/thought_proj...


In [27]:
df7_bot_comment = "Thanks for posting on /r/bipolar!      Please take a second to [read our rules](/r/bipolar/about/rules); if you haven't already, make sure that your post **does not** have any personal information (including your name/signature/tag on art).  *^(A moderator has not removed your submission; this is not a punitive action. We intend this comment solely to be informative.)*  --- Community News  - &#127883; [Want to join the Mod Team?](https://www.reddit.com/r/bipolar/comments/112z7ps/mod_applications_are_open/)  - &#127908; See our [Community Discussion](https://www.reddit.com/r/bipolar/about/sticky) - Desktop or Desktop mode on a mobile device.  - &#127969; If you are open to answering questions from those that live with a loved one diagnosed with Bipolar Disorder, please see r/family_of_bipolar.  Thank you for participating!   *I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/bipolar) if you have any questions or concerns.*"
df7 = df7[df7['post_comment'] != df7_bot_comment]
df7

,post_title,post_content,post_comment,subreddit,sentiment,post_date,post_link,comment_date,comment_link
1,I took a shower,You know I am trying to figure out why a 65 ye...,yeah i hate showering and have no reason to ha...,bipolar,positive,2024-03-04 21:51:50,https://www.reddit.com/r/bipolar/comments/1b6n...,2024-03-05 00:25:58,/r/bipolar/comments/1b6nbk0/i_took_a_shower/kt...
2,I took a shower,You know I am trying to figure out why a 65 ye...,Reminded me of a [post](https://www.reddit.com...,bipolar,positive,2024-03-04 21:51:50,https://www.reddit.com/r/bipolar/comments/1b6n...,2024-03-05 02:54:03,/r/bipolar/comments/1b6nbk0/i_took_a_shower/kt...
3,I took a shower,You know I am trying to figure out why a 65 ye...,I hear you! As soon as I saw the title I wante...,bipolar,positive,2024-03-04 21:51:50,https://www.reddit.com/r/bipolar/comments/1b6n...,2024-03-05 03:26:54,/r/bipolar/comments/1b6nbk0/i_took_a_shower/kt...
4,I took a shower,You know I am trying to figure out why a 65 ye...,Maybe try to make it less of a chore and more ...,bipolar,positive,2024-03-04 21:51:50,https://www.reddit.com/r/bipolar/comments/1b6n...,2024-03-05 03:18:37,/r/bipolar/comments/1b6nbk0/i_took_a_shower/kt...
5,I took a shower,You know I am trying to figure out why a 65 ye...,I hate showering too and also don’t know why. ...,bipolar,positive,2024-03-04 21:51:50,https://www.reddit.com/r/bipolar/comments/1b6n...,2024-03-05 04:45:36,/r/bipolar/comments/1b6nbk0/i_took_a_shower/kt...
...,...,...,...,...,...,...,...,...,...
880,Do you have issues switching medications?,I tried to switch medications and I freaked ou...,Anytime you switch meds there’s chance for iss...,bipolar,negative,2024-02-22 05:10:24,https://www.reddit.com/r/bipolar/comments/1awy...,2024-02-22 05:33:14,/r/bipolar/comments/1awy7vk/do_you_have_issues...
882,Dead eyes,Looking at old photos I took in an attempt to ...,Dead eyes is how I know I’m depressed it’s kin...,bipolar,negative,2024-02-21 23:43:17,https://www.reddit.com/r/bipolar/comments/1awr...,2024-02-22 06:44:53,/r/bipolar/comments/1awrbp1/dead_eyes/krkp9zc/
883,Dead eyes,Looking at old photos I took in an attempt to ...,"Funny you say this, my husband makes comments ...",bipolar,positive,2024-02-21 23:43:17,https://www.reddit.com/r/bipolar/comments/1awr...,2024-02-22 02:00:21,/r/bipolar/comments/1awrbp1/dead_eyes/krjon1f/
885,Two years since hit and run,So back in 2022 I crawled under a red crv caus...,"Dawg, that is insane levels of mania. Kudos to...",bipolar,positive,2024-02-21 21:46:04,https://www.reddit.com/r/bipolar/comments/1awo...,2024-02-21 22:52:58,/r/bipolar/comments/1awodf2/two_years_since_hi...


In [28]:
df8_bot_comment = "*r/ptsd has generated this automated response that is appended to every post*  Welcome to r/ptsd! We are a supportive & respectful community. If you realise that your post is in conflict with our rules (and is in risk of being removed), you are welcome to edit your post.  You do not have to delete it.  As a reminder: never post or share personal contact information. Traumatized people are often distracted, desperate for a personal connection, so may be more vulnerable to lurking or past abusers, trolls, phishing, or other scams. *Your safety always comes first!*  If you are offering help, you may also end up doing more damage by offering to support somebody privately. Reddit explains why: [Do NOT exchange DMs or personal info with anyone you don't know!](https://www.reddit.com/r/SWResources/comments/dmu24/why_shouldnt_i_share_my_contact_information/)  If you or someone you know is in immediate danger, please contact your GP/doctor, go to A&E/hospital, or call your emergency services number.  Reddit list: [US and global, multilingual suicide and support hotlines](https://www.reddit.com/r/SuicideWatch/wiki/hotlines).  Suicide is not a forbidden word, but please do not include depictions or methods of suicide in your post.   And as a friendly reminder, PTSD is an equal opportunity disorder. PTSD does not discriminate. And neither do we. Gatekeeping is not allowed here.  *I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/ptsd) if you have any questions or concerns.*"
df8 = df8[df8['post_comment'] != df8_bot_comment]
df8

,post_title,post_content,post_comment,subreddit,sentiment,post_date,post_link,comment_date,comment_link
1,DAE have somatic flashbacks?,Maybe an odd post but I've noticed a major sym...,Jesus somatic flashbacks are my worst symptom ...,ptsd,positive,2024-03-04 19:42:25,https://www.reddit.com/r/ptsd/comments/1b6k3jx...,2024-03-04 21:54:08,/r/ptsd/comments/1b6k3jx/dae_have_somatic_flas...
2,DAE have somatic flashbacks?,Maybe an odd post but I've noticed a major sym...,Surprisingly physical flashbacks were my first...,ptsd,positive,2024-03-04 19:42:25,https://www.reddit.com/r/ptsd/comments/1b6k3jx...,2024-03-05 01:34:41,/r/ptsd/comments/1b6k3jx/dae_have_somatic_flas...
5,"it's never going to get better, is it ?",tw sexual assault The traumatic events that ga...,Hey. I’m not sure where you’re from but I know...,ptsd,positive,2024-03-04 15:45:55,https://www.reddit.com/r/ptsd/comments/1b6e3tg...,2024-03-04 16:51:03,/r/ptsd/comments/1b6e3tg/its_never_going_to_ge...
6,"it's never going to get better, is it ?",tw sexual assault The traumatic events that ga...,I’m also suffering from PTSD. It’s infuriating...,ptsd,positive,2024-03-04 15:45:55,https://www.reddit.com/r/ptsd/comments/1b6e3tg...,2024-03-04 18:18:12,/r/ptsd/comments/1b6e3tg/its_never_going_to_ge...
8,Trying to understand my trauma,A little bit of context. When i was young age ...,Yes I think it's normal. A psychologist told ...,ptsd,negative,2024-03-04 16:04:04,https://www.reddit.com/r/ptsd/comments/1b6ekk3...,2024-03-04 18:52:17,/r/ptsd/comments/1b6ekk3/trying_to_understand_...
...,...,...,...,...,...,...,...,...,...
1064,I still have PTSD from high school despite gra...,I was bullied to hell and back. School did no...,I can relate. A large chunk of my trauma is fr...,ptsd,negative,2024-01-27 02:06:30,https://www.reddit.com/r/ptsd/comments/1abzndy...,2024-01-27 02:34:56,/r/ptsd/comments/1abzndy/i_still_have_ptsd_fro...
1065,I still have PTSD from high school despite gra...,I was bullied to hell and back. School did no...,"that sounds terrible, it’s like psychological ...",ptsd,negative,2024-01-27 02:06:30,https://www.reddit.com/r/ptsd/comments/1abzndy...,2024-01-27 02:22:56,/r/ptsd/comments/1abzndy/i_still_have_ptsd_fro...
1066,I still have PTSD from high school despite gra...,I was bullied to hell and back. School did no...,I could’ve written the first paragraph - scary...,ptsd,positive,2024-01-27 02:06:30,https://www.reddit.com/r/ptsd/comments/1abzndy...,2024-01-27 17:48:57,/r/ptsd/comments/1abzndy/i_still_have_ptsd_fro...
1067,I still have PTSD from high school despite gra...,I was bullied to hell and back. School did no...,Have you considered working at clearing the pt...,ptsd,positive,2024-01-27 02:06:30,https://www.reddit.com/r/ptsd/comments/1abzndy...,2024-01-27 16:04:01,/r/ptsd/comments/1abzndy/i_still_have_ptsd_fro...


In [29]:
reddit_post_with_comments = pd.concat([df1, df2, df4, df4, df5, df6, df7, df8, df9, df10])
reddit_post_with_comments = reddit_post_with_comments.reset_index(drop=True)
reddit_post_with_comments

,post_title,post_content,post_comment,subreddit,sentiment,post_date,post_link,comment_date,comment_link
0,How do I get over sexual jealousy,I know how pathetic it sounds but I really don...,1. Don't normalize what other people do or wha...,mentalhealth,negative,2024-03-04 23:10:43,https://www.reddit.com/r/mentalhealth/comments...,2024-03-04 23:44:26,/r/mentalhealth/comments/1b6pc2l/how_do_i_get_...
1,How do I get over sexual jealousy,I know how pathetic it sounds but I really don...,Never compare yourself to anybody else. Your l...,mentalhealth,negative,2024-03-04 23:10:43,https://www.reddit.com/r/mentalhealth/comments...,2024-03-05 00:28:51,/r/mentalhealth/comments/1b6pc2l/how_do_i_get_...
2,How do I get over sexual jealousy,I know how pathetic it sounds but I really don...,I can relate but I do like to remind myself th...,mentalhealth,negative,2024-03-04 23:10:43,https://www.reddit.com/r/mentalhealth/comments...,2024-03-04 23:46:40,/r/mentalhealth/comments/1b6pc2l/how_do_i_get_...
3,How do I get over sexual jealousy,I know how pathetic it sounds but I really don...,I'm almost certain most people lie about the a...,mentalhealth,positive,2024-03-04 23:10:43,https://www.reddit.com/r/mentalhealth/comments...,2024-03-05 00:02:33,/r/mentalhealth/comments/1b6pc2l/how_do_i_get_...
4,How do I get over sexual jealousy,I know how pathetic it sounds but I really don...,Find yourself a nice woman and try to have a r...,mentalhealth,negative,2024-03-04 23:10:43,https://www.reddit.com/r/mentalhealth/comments...,2024-03-05 00:05:27,/r/mentalhealth/comments/1b6pc2l/how_do_i_get_...
...,...,...,...,...,...,...,...,...,...
16940,Thought Projection,"Hello, as a Schizophrenic one of my only illus...",For almost a year I thought my partner was in ...,schizophrenia,positive,2024-02-15 00:23:45,https://www.reddit.com/r/schizophrenia/comment...,2024-02-15 02:51:35,/r/schizophrenia/comments/1ar2e2a/thought_proj...
16941,Thought Projection,"Hello, as a Schizophrenic one of my only illus...","One thing that could be helpful, is if people ...",schizophrenia,positive,2024-02-15 00:23:45,https://www.reddit.com/r/schizophrenia/comment...,2024-02-17 02:47:02,/r/schizophrenia/comments/1ar2e2a/thought_proj...
16942,Thought Projection,"Hello, as a Schizophrenic one of my only illus...",[removed],schizophrenia,positive,2024-02-15 00:23:45,https://www.reddit.com/r/schizophrenia/comment...,2024-02-15 00:34:53,/r/schizophrenia/comments/1ar2e2a/thought_proj...
16943,Thought Projection,"Hello, as a Schizophrenic one of my only illus...",I have that problem too- it’s just what it is ...,schizophrenia,positive,2024-02-15 00:23:45,https://www.reddit.com/r/schizophrenia/comment...,2024-02-15 07:59:11,/r/schizophrenia/comments/1ar2e2a/thought_proj...


In [30]:
#Sentiment count for reddit_post_with_comments
sentiment_counts_reddit_posts_comments = reddit_post_with_comments['sentiment'].value_counts()
sentiment_counts_reddit_posts_comments

positive    10053
negative     6624
neutral       268
Name: sentiment, dtype: int64

In [31]:
reddit_post_with_comments.to_csv('reddit_post_with_separated_comments_with_datetime.csv', index=False)

In [ ]:
reddit_post_df_comments_only = reddit_post_with_comments[['post_comment', 'subreddit', 'sentiment']].copy()
reddit_post_df_comments_only

,post_comment,subreddit,sentiment
0,Its never to late to swtich professions if you...,mentalhealth,positive
1,I've been in my current tech management role f...,mentalhealth,positive
2,Choose a specialty with a relaxed/easier train...,mentalhealth,positive
3,I changed careers twice. First one was simply ...,mentalhealth,positive
4,"This sounds like me, except veterinary! I don'...",mentalhealth,negative
...,...,...,...
17150,I have a female protector spirit voice on the ...,schizophrenia,positive
17151,happy sunday!!,schizophrenia,positive
17152,Great pic and thanks for the kind message. Kee...,schizophrenia,positive
17153,I've been listening to this with my morning co...,schizophrenia,positive


In [ ]:
#recalculating sentiment solely based on post_comments only
reddit_post_df_comments_only['sentiment'] = reddit_post_df_comments_only['post_comment'].apply(get_sentiment)
reddit_post_df_comments_only

,post_comment,subreddit,sentiment
0,Its never to late to swtich professions if you...,mentalhealth,positive
1,I've been in my current tech management role f...,mentalhealth,positive
2,Choose a specialty with a relaxed/easier train...,mentalhealth,positive
3,I changed careers twice. First one was simply ...,mentalhealth,positive
4,"This sounds like me, except veterinary! I don'...",mentalhealth,negative
...,...,...,...
17150,I have a female protector spirit voice on the ...,schizophrenia,positive
17151,happy sunday!!,schizophrenia,positive
17152,Great pic and thanks for the kind message. Kee...,schizophrenia,positive
17153,I've been listening to this with my morning co...,schizophrenia,positive


In [ ]:
sentiment_counts_reddit_posts_comments_only = reddit_post_df_comments_only['sentiment'].value_counts()
sentiment_counts_reddit_posts_comments_only

positive    9679
negative    4462
neutral     3014
Name: sentiment, dtype: int64

In [ ]:
reddit_post_df_comments_only.to_csv('reddit_post_with_comments_only.csv', index=False)